In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1jACsP0mL4_YTew5Kgn2Z9K-lgjFDsGnq", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

# Self-Attention on Image Patches: How Patches Learn to Talk to Each Other

*Part 2 of the Vizuara series on Vision Transformers from Scratch*

*Estimated time: 60 minutes*

In Notebook 1 we learned how to chop an image into patches and project them into an embedding space. But those patch tokens are still strangers to each other — each one knows only about the small rectangle of pixels it came from. Self-attention is the mechanism that lets patches communicate, compare, and build a shared understanding of the entire image.

By the end of this notebook, you will have built every component of a Transformer encoder block from scratch — and you will **see** attention in action through heatmaps that reveal which patches are paying attention to which.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/vision-transformers-from-scratch/practice/2/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 1: Why Does This Matter?

Let us take stock of where we are in our journey.

In **Notebook 1**, we turned a 32x32 CIFAR-10 image into a sequence of 64 patch tokens (using 4x4 patches), each represented as a 128-dimensional vector. We also prepended a special `[CLS]` token and added positional embeddings, giving us a sequence of shape `(65, 128)`.

But right now, **each patch is an island**. Patch 17 knows it contains some brown fur texture, but it has no idea that Patch 18 next to it also has fur, or that Patch 42 far away contains a dog's ear. The patches cannot see each other.

**Self-attention changes everything.** It is the mechanism by which each patch broadcasts a question to every other patch: *"Hey, how relevant are you to me?"* Based on the answers, each patch builds a richer, context-aware representation that incorporates information from the entire image.

By the end of this notebook, we will:
- Understand the Query-Key-Value mechanism intuitively and mathematically
- Build scaled dot-product attention from raw matrix multiplications
- Implement multi-head attention, the MLP block, and the full Transformer encoder block
- Visualize attention heatmaps showing which patches attend to which — and watch attention patterns become more semantic in deeper layers

In [ ]:
# Let us set up our environment right away
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
np.random.seed(42)

print("All imports ready!")
print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
#@title 🎧 Listen: Conference Analogy
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_conference_analogy.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 2: Building Intuition

Before we touch any code or equations, let us build a strong mental model for what self-attention does.

### The Conference Analogy

Imagine 65 people standing in a conference hall. Each person holds one piece of a jigsaw puzzle (one image patch). Right now, each person can only see their own piece — they have no idea what the full picture looks like.

**Self-attention is the networking session.** Each person walks around the room and asks every other person: *"How relevant is your piece to mine?"* The person holding a dog's face piece will find that the person holding the dog's body piece is highly relevant, while the person holding a blue sky piece is less so.

After this round of conversations, each person updates their understanding. The dog-face person now thinks: *"I'm a dog face, and I'm connected to a body below me and an ear to my left."* They have gone from knowing about a 4x4 pixel patch to understanding their role in the bigger picture.

In [ ]:
#@title 🎧 Listen: Qkv Framework
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_qkv_framework.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### The Query-Key-Value Framework

Self-attention uses three learned projections — Query, Key, and Value — and it helps to understand *why* we need three separate things rather than one.

- **Query (Q)** = *"What am I looking for?"* — Like typing a search query into Google. A patch of a dog's face might be looking for body parts, other facial features, or context clues.

- **Key (K)** = *"What do I contain?"* — Like the title or metadata of a webpage. A patch describes itself so others can find it. The dog-body patch advertises: "I contain fur and torso-like structure."

- **Value (V)** = *"Here is my actual content."* — Like the full text of the webpage. Once a match is found between Query and Key, the Value is what actually gets passed along.

**Why three matrices, not one?** Because the question *"What am I looking for?"* is fundamentally different from *"What do I contain."* A patch of a dog's face might **look for** body and leg patches (that is its Query), but **describe itself** as containing fur texture and an eye (that is its Key). These are different roles that require different learned projections.

### Think About This

*In natural language, the word "it" needs to attend to different words in the sentence to figure out what "it" refers to. What is the equivalent situation in images? When would one patch need to "look at" a distant patch to understand itself?*

Think about an image of a person throwing a ball. The patch containing the hand needs to attend to the ball patch (even though they may be far apart) to understand the action. The patch containing a shadow on the ground needs to attend to the object casting that shadow. Self-attention gives patches this ability to reach across the image and gather relevant context.

In [ ]:
#@title 🎧 Listen: Quadratic Cost
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_quadratic_cost.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# Quick thought experiment: how many patch-to-patch comparisons
# does self-attention make?
num_patches = 64 + 1  # 64 image patches + 1 CLS token
comparisons = num_patches * num_patches
print(f"With {num_patches} tokens, self-attention computes:")
print(f"  {comparisons:,} pairwise attention scores")
print(f"  That is a {num_patches}x{num_patches} attention matrix!")
print(f"\nThis is why self-attention is O(N^2) — and why")
print(f"efficient attention variants are an active research area.")

In [ ]:
#@title 🎧 Listen: Math Qkv
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_math_qkv.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 3: The Mathematics

Now let us formalize the intuition with precise mathematics. After every equation, we will explain exactly what the computer is doing.

### 3.1 Setup

Let us say we have a sequence of $N$ patch tokens, each of dimension $d$. We stack them into a matrix $z \in \mathbb{R}^{N \times d}$.

### 3.2 Query, Key, Value Projections

$$Q = zW^Q, \quad K = zW^K, \quad V = zW^V$$

where $W^Q, W^K, W^V \in \mathbb{R}^{d \times d_k}$.

**What the computer does:** Take our patch embeddings $z$ (shape $N \times d$) and multiply by three separate weight matrices. This gives each token three different "views" of itself — a query view, a key view, and a value view. Each view has dimension $d_k$ (typically $d_k = d / h$ where $h$ is the number of attention heads).

In [ ]:
#@title 🎧 Listen: Math Multihead Block
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_math_multihead_block.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 3.3 Attention Scores and Weights

$$\text{scores} = \frac{QK^T}{\sqrt{d_k}}$$

**What the computer does:** Compute the dot product of every query with every key. The result is an $N \times N$ matrix where entry $(i, j)$ measures how much token $i$'s query matches token $j$'s key — i.e., how relevant token $j$ is to token $i$. We divide by $\sqrt{d_k}$ to prevent the scores from becoming too large. When $d_k$ is large, dot products tend to grow in magnitude, which pushes softmax into regions where it produces near-one-hot outputs. This kills gradients and makes training unstable.

$$\alpha = \text{softmax}(\text{scores})$$

**What the computer does:** Apply softmax along each row, converting raw scores into a probability distribution. After this, each row of $\alpha$ sums to 1. Entry $\alpha_{ij}$ tells us what fraction of its attention token $i$ pays to token $j$.

### 3.4 Attention Output

$$\text{Attention}(Q, K, V) = \alpha V$$

**What the computer does:** For each token $i$, compute a weighted sum of all value vectors, where the weights are the attention probabilities. If patch $i$ strongly attends to patch $j$ (large $\alpha_{ij}$), then the output for patch $i$ will be heavily influenced by patch $j$'s value vector.

### 3.5 Multi-Head Attention

$$\text{MultiHead}(z) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) \, W^O$$

where each $\text{head}_i = \text{Attention}(zW_i^Q, zW_i^K, zW_i^V)$.

**What the computer does:** Run $h$ independent attention operations in parallel, each operating on a $d_k$-dimensional slice of the embedding. Then concatenate all the outputs (giving us back dimension $d$) and project through one final matrix $W^O$. Each head can learn a *different* attention pattern — one head might focus on spatial neighbors, another on color similarity, another on semantic relationships.

### 3.6 The Transformer Block

$$z'_l = \text{MHSA}(\text{LN}(z_{l-1})) + z_{l-1}$$
$$z_l = \text{MLP}(\text{LN}(z'_l)) + z'_l$$

**What the computer does:** This is the pre-norm Transformer block. First, apply LayerNorm, then multi-head self-attention, then add the input back (residual connection). Then apply LayerNorm again, pass through an MLP, and add back again. The residual connections are critical — they keep gradients flowing through deep networks and let each layer learn *incremental refinements* rather than full transformations.

### 3.7 The MLP

$$\text{MLP}(z) = \text{GELU}(zW_1 + b_1)W_2 + b_2$$

where $W_1 \in \mathbb{R}^{d \times 4d}$ and $W_2 \in \mathbb{R}^{4d \times d}$.

**What the computer does:** Expand to 4x the dimension (e.g., 128 to 512), apply GELU activation (a smooth variant of ReLU), then compress back to the original dimension. This gives each token a chance to process information through a much higher-dimensional space — like temporarily unfolding a piece of paper to write more notes before folding it back up.

In [ ]:
# Quick check: let us see the dimensions we will be working with
print("Our ViT configuration for this notebook:")
print(f"  Image: 32x32, Patch: 4x4")
print(f"  Number of patches: {(32//4)**2} = 64")
print(f"  Sequence length (with CLS): 65")
print(f"  Embedding dim (d): 128")
print(f"  Num heads (h): 4")
print(f"  Head dim (d_k = d/h): {128//4}")
print(f"  MLP hidden dim (4*d): {4*128}")

In [ ]:
#@title 🎧 Listen: Toy Example
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_toy_example.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 3.8 Concrete Toy Example

Let us work through attention with 3 patches and $d_k = 2$ to see every step.

| Patch | Meaning | Query | Key | Value |
|-------|---------|-------|-----|-------|
| Patch 1 | Dog face | [1.0, 0.0] | [1.0, 0.0] | [0.9, 0.1] |
| Patch 2 | Dog body | [0.8, 0.2] | [0.9, 0.1] | [0.7, 0.3] |
| Patch 3 | Sky | [0.0, 1.0] | [0.1, 0.9] | [0.2, 0.8] |

**Step 1: Compute scores** $= QK^T / \sqrt{d_k}$

$QK^T = \begin{bmatrix} 1\cdot1 + 0\cdot0 & 1\cdot0.9 + 0\cdot0.1 & 1\cdot0.1 + 0\cdot0.9 \\ 0.8\cdot1 + 0.2\cdot0 & 0.8\cdot0.9 + 0.2\cdot0.1 & 0.8\cdot0.1 + 0.2\cdot0.9 \\ 0\cdot1 + 1\cdot0 & 0\cdot0.9 + 1\cdot0.1 & 0\cdot0.1 + 1\cdot0.9 \end{bmatrix} = \begin{bmatrix} 1.0 & 0.9 & 0.1 \\ 0.8 & 0.74 & 0.26 \\ 0.0 & 0.1 & 0.9 \end{bmatrix}$

Dividing by $\sqrt{2} \approx 1.414$:

$\text{scores} = \begin{bmatrix} 0.707 & 0.636 & 0.071 \\ 0.566 & 0.523 & 0.184 \\ 0.0 & 0.071 & 0.636 \end{bmatrix}$

**Step 2: Softmax** (per row)

Row 1: $[0.707, 0.636, 0.071] \rightarrow e^{0.707} = 2.028, \; e^{0.636} = 1.889, \; e^{0.071} = 1.074 \rightarrow$ sum $= 4.991$
$\alpha_1 = [0.406, 0.379, 0.215]$

The dog face attends most to itself (0.406), strongly to the dog body (0.379), and least to the sky (0.215). This makes sense — the dog face is most similar to itself and the dog body.

**Step 3: Weighted sum of values**

Output for Patch 1 = $0.406 \times [0.9, 0.1] + 0.379 \times [0.7, 0.3] + 0.215 \times [0.2, 0.8]$
$= [0.365, 0.041] + [0.265, 0.114] + [0.043, 0.172]$
$= [0.673, 0.327]$

The output has been enriched — it is no longer just the dog face's value, but a blend weighted toward semantically similar patches.

Let us now verify this computation in code.

### Verifying the Toy Example

Let us confirm our hand calculations with PyTorch.

In [ ]:
# Our toy example: 3 patches, d_k = 2
Q = torch.tensor([[1.0, 0.0],    # Dog face query
                   [0.8, 0.2],    # Dog body query
                   [0.0, 1.0]])   # Sky query

K = torch.tensor([[1.0, 0.0],    # Dog face key
                   [0.9, 0.1],    # Dog body key
                   [0.1, 0.9]])   # Sky key

V = torch.tensor([[0.9, 0.1],    # Dog face value
                   [0.7, 0.3],    # Dog body value
                   [0.2, 0.8]])   # Sky value

d_k = Q.shape[-1]

In [ ]:
# Step 1: Compute scaled scores
scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
print("Scores (QK^T / sqrt(d_k)):")
print(scores)

# Step 2: Softmax
attn_weights = F.softmax(scores, dim=-1)
print("\nAttention weights (softmax of scores):")
print(attn_weights)

# Step 3: Weighted sum of values
output = torch.matmul(attn_weights, V)
print("\nAttention output:")
print(output)

In [ ]:
#@title 🎧 Listen: Toy Heatmap
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_toy_heatmap.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us visualize these attention weights as a heatmap.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
labels = ['Dog Face', 'Dog Body', 'Sky']

im = ax.imshow(attn_weights.detach().numpy(), cmap='Blues', vmin=0, vmax=1)
ax.set_xticks(range(3))
ax.set_yticks(range(3))
ax.set_xticklabels(labels, fontsize=11)
ax.set_yticklabels(labels, fontsize=11)
ax.set_xlabel('Attends TO (Keys)', fontsize=12)
ax.set_ylabel('Attends FROM (Queries)', fontsize=12)
ax.set_title('Attention Weights — Toy Example', fontsize=13, fontweight='bold')

# Annotate cells with values
for i in range(3):
    for j in range(3):
        val = attn_weights[i, j].item()
        color = 'white' if val > 0.5 else 'black'
        ax.text(j, i, f'{val:.3f}', ha='center', va='center',
                fontsize=13, fontweight='bold', color=color)

plt.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.show()

Notice how the dog face attends strongly to both itself and the dog body, but weakly to the sky. The sky patch, conversely, attends strongly to itself and weakly to the dog patches. This is exactly the behavior we want — semantically related patches find each other.

In [ ]:
#@title 🎧 Listen: Building Attention
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_building_attention.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 4: Let Us Build It — Component by Component

Now we build every component of the Transformer encoder from scratch, starting from the lowest level and working up. No black-box `nn.MultiheadAttention` — we will implement everything with raw matrix multiplications so we understand every single operation.

### 4.1 Scaled Dot-Product Attention (from scratch)

This is the core primitive. Everything else is built on top of it.

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Compute scaled dot-product attention from scratch.
    Q, K: (..., seq_len, d_k)  V: (..., seq_len, d_v)
    Returns: output (..., seq_len, d_v), attn_weights (..., seq_len, seq_len)
    """
    d_k = Q.shape[-1]

    # Step 1: Compute raw attention scores
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

    # Step 2: Apply mask if provided (for causal attention)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))

    # Step 3: Softmax to get attention weights
    attn_weights = F.softmax(scores, dim=-1)

    # Step 4: Weighted sum of values
    output = torch.matmul(attn_weights, V)

    return output, attn_weights

print("scaled_dot_product_attention() defined!")

Let us verify our function matches the toy example.

In [ ]:
output, weights = scaled_dot_product_attention(Q, K, V)
print("Our function output:")
print(output)
print("\nMatches our hand calculation? ",
      torch.allclose(output, torch.matmul(F.softmax(
          torch.matmul(Q, K.T) / math.sqrt(2), dim=-1), V), atol=1e-6))

In [ ]:
#@title 🎧 Listen: Attention Head
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_attention_head.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Single Attention Head

Now we wrap the attention computation inside a learnable module. An attention head learns its own $W^Q$, $W^K$, $W^V$ matrices.

In [ ]:
class AttentionHead(nn.Module):
    """
    A single attention head with learnable Q, K, V projections.

    Args:
        embed_dim: Input embedding dimension
        head_dim: Dimension of this head (typically embed_dim // num_heads)
    """
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.W_Q = nn.Linear(embed_dim, head_dim, bias=False)
        self.W_K = nn.Linear(embed_dim, head_dim, bias=False)
        self.W_V = nn.Linear(embed_dim, head_dim, bias=False)

    def forward(self, x):
        Q = self.W_Q(x)  # (batch, seq_len, head_dim)
        K = self.W_K(x)
        V = self.W_V(x)
        output, attn_weights = scaled_dot_product_attention(Q, K, V)
        return output, attn_weights

print("AttentionHead class defined!")

Let us test a single attention head with some random patch embeddings.

In [ ]:
# Simulate: batch=1, seq_len=65 (64 patches + CLS), embed_dim=128
batch_size = 1
seq_len = 65   # 64 patches + 1 CLS token
embed_dim = 128
head_dim = 32  # Will have 4 heads: 128 / 4 = 32

x = torch.randn(batch_size, seq_len, embed_dim)

head = AttentionHead(embed_dim, head_dim)
output, attn_weights = head(x)

print(f"Input shape:            {x.shape}")
print(f"Output shape:           {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"\nEach row sums to: {attn_weights[0, 0].sum().item():.4f}")

In [ ]:
#@title 🎧 Listen: Attention Visualization
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_attention_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Let us visualize what this random attention head produces. Even with untrained weights, we can see the structure of the attention matrix.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full attention matrix
ax = axes[0]
im = ax.imshow(attn_weights[0].detach().numpy(), cmap='viridis', aspect='auto')
ax.set_xlabel('Key position', fontsize=11)
ax.set_ylabel('Query position', fontsize=11)
ax.set_title('Full Attention Matrix (65x65)\nSingle Untrained Head',
             fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax, shrink=0.8)

In [ ]:
# CLS token attention (row 0) — what does the CLS token attend to?
fig, ax = plt.subplots(figsize=(10, 3))
cls_attn = attn_weights[0, 0].detach().numpy()
ax.bar(range(len(cls_attn)), cls_attn, color='steelblue', alpha=0.8)
ax.set_xlabel('Token position', fontsize=11)
ax.set_ylabel('Attention weight', fontsize=11)
ax.set_title('[CLS] Token Attention Distribution\n(Untrained — roughly uniform)',
             fontsize=12, fontweight='bold')
ax.axhline(y=1/seq_len, color='red', linestyle='--',
           alpha=0.7, label=f'Uniform: 1/{seq_len}')
ax.legend()
plt.tight_layout()
plt.show()

With untrained weights, the attention is roughly uniform — the CLS token pays about equal attention to every patch. After training, this will become much more focused and semantically meaningful.

In [ ]:
#@title 🎧 Listen: Multihead
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_multihead.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.3 Multi-Head Attention

This is where the magic happens. Instead of one attention head with a single perspective, we run multiple heads in parallel. Each head can learn to focus on different types of relationships — one might track spatial proximity, another color similarity, another shape correspondence.

In [ ]:
class MultiHeadAttention(nn.Module):
    """Multi-Head Self-Attention from scratch."""
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList([
            AttentionHead(embed_dim, self.head_dim)
            for _ in range(num_heads)])
        self.W_O = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        head_outputs, all_attn_weights = [], []
        for head in self.heads:
            out, attn_w = head(x)
            head_outputs.append(out)
            all_attn_weights.append(attn_w)
        concatenated = torch.cat(head_outputs, dim=-1)
        output = self.W_O(concatenated)
        return output, all_attn_weights

print("MultiHeadAttention class defined!")

Let us test multi-head attention and inspect the outputs.

In [ ]:
mhsa = MultiHeadAttention(embed_dim=128, num_heads=4)
x = torch.randn(1, 65, 128)
output, all_attn_weights = mhsa(x)

print(f"Input shape:  {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Number of heads: {len(all_attn_weights)}")
print(f"Each head's attention shape: {all_attn_weights[0].shape}")

Now let us visualize the attention patterns from all 4 heads side by side. Even with random weights, each head will have a slightly different pattern because of their independent weight initializations.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Attention Patterns from 4 Different Heads\n'
             '(Untrained — each head has unique random patterns)',
             fontsize=14, fontweight='bold', y=1.02)

for idx, ax in enumerate(axes.flat):
    attn = all_attn_weights[idx][0].detach().numpy()
    im = ax.imshow(attn, cmap='inferno', aspect='auto')
    ax.set_title(f'Head {idx + 1}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Key position')
    ax.set_ylabel('Query position')
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.tight_layout()
plt.show()

Each head produces a different attention matrix. Once trained, these heads specialize: some might focus on local neighborhoods (like a convolution), others on long-range semantic relationships, and others on global structural patterns. This diversity is a key advantage of multi-head attention over single-head.

In [ ]:
#@title 🎧 Listen: Mlp Gelu
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_mlp_gelu.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.4 The MLP Block

After the attention layer mixes information *across* tokens, the MLP processes each token *independently*. It expands the representation to a higher dimension (4x), applies a nonlinearity, and compresses back.

In [ ]:
class MLP(nn.Module):
    """
    MLP block for the Transformer encoder.
    Two-layer feedforward network with GELU and 4x expansion.
    """
    def __init__(self, embed_dim, mlp_ratio=4, dropout=0.0):
        super().__init__()
        hidden_dim = int(embed_dim * mlp_ratio)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)        # (batch, seq_len, hidden_dim)
        x = self.act(x)        # GELU activation
        x = self.dropout(x)
        x = self.fc2(x)        # (batch, seq_len, embed_dim)
        x = self.dropout(x)
        return x

print("MLP class defined!")
print(f"For embed_dim=128: input -> 128 -> {128*4} -> 128")

Let us quickly compare GELU to ReLU so we understand why modern Transformers prefer GELU.

In [ ]:
x_range = torch.linspace(-4, 4, 200)
relu_out = F.relu(x_range)
gelu_out = F.gelu(x_range)

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(x_range.numpy(), relu_out.numpy(),
        label='ReLU', linewidth=2, color='coral')
ax.plot(x_range.numpy(), gelu_out.numpy(),
        label='GELU', linewidth=2, color='steelblue')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Input', fontsize=12)
ax.set_ylabel('Output', fontsize=12)
ax.set_title('GELU vs ReLU', fontsize=13, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

GELU is smooth everywhere (no sharp corner at 0) and slightly suppresses small negative values rather than zeroing them out. This smoothness helps with gradient flow during training.

In [ ]:
#@title 🎧 Listen: Transformer Block
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_transformer_block.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.5 Full Transformer Encoder Block

Now we combine everything: LayerNorm, Multi-Head Self-Attention, the MLP, and residual connections. This is the complete building block that gets stacked to create the full Transformer encoder.

In [ ]:
class TransformerBlock(nn.Module):
    """
    A single Transformer encoder block (pre-norm variant).
    x -> LN -> MHSA -> +residual -> LN -> MLP -> +residual -> out
    """
    def __init__(self, embed_dim, num_heads, mlp_ratio=4, dropout=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mhsa = MultiHeadAttention(embed_dim, num_heads)
        self.mlp = MLP(embed_dim, mlp_ratio, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Sub-block 1: LayerNorm -> MHSA -> Residual
        normed = self.norm1(x)
        attn_output, attn_weights = self.mhsa(normed)
        x = x + self.dropout(attn_output)

        # Sub-block 2: LayerNorm -> MLP -> Residual
        normed = self.norm2(x)
        mlp_output = self.mlp(normed)
        x = x + self.dropout(mlp_output)

        return x, attn_weights

print("TransformerBlock class defined!")

Let us test it and verify the shapes.

In [ ]:
block = TransformerBlock(embed_dim=128, num_heads=4)
x = torch.randn(1, 65, 128)  # batch=1, 65 tokens, dim=128

output, attn_weights = block(x)

print(f"Input shape:  {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Shape preserved: {x.shape == output.shape}")
print(f"Number of attention heads: {len(attn_weights)}")

# Verify residual connection
diff = (output - x).abs().mean().item()
print(f"\nMean absolute diff from input: {diff:.4f}")
print("(Non-zero but moderate — residual connection is working!)")

In [ ]:
#@title 🎧 Listen: Todo1
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_todo1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 5: Your Turn

Time to test your understanding! Below are three exercises — the first two are essential, the third is a stretch goal.

### TODO 1: Implement Scaled Dot-Product Attention from Scratch

Without looking at our earlier implementation, write `scaled_dot_product_attention_v2`. We provide the function signature and step-by-step hints.

In [ ]:
def scaled_dot_product_attention_v2(Q, K, V):
    """
    Implement scaled dot-product attention from scratch.

    Args:
        Q: Query tensor of shape (batch, seq_len, d_k)
        K: Key tensor of shape (batch, seq_len, d_k)
        V: Value tensor of shape (batch, seq_len, d_v)

    Returns:
        output: Shape (batch, seq_len, d_v)
        attn_weights: Shape (batch, seq_len, seq_len)

    Steps:
        1. Get d_k from the last dimension of Q
        2. Compute scores = Q @ K^T, scale by 1/sqrt(d_k)
        3. Apply softmax along the last dimension
        4. Multiply attention weights by V
        5. Return both output and attention weights
    """
    # TODO: Your implementation here (~5 lines)
    # d_k = ...
    # scores = ...
    # attn_weights = ...
    # output = ...
    # return output, attn_weights
    pass

**Verification cell** — run this after you implement the function above.

In [ ]:
# Verification for TODO 1
torch.manual_seed(123)
Q_test = torch.randn(2, 10, 32)
K_test = torch.randn(2, 10, 32)
V_test = torch.randn(2, 10, 32)

your_output, your_weights = scaled_dot_product_attention_v2(
    Q_test, K_test, V_test)
ref_output, ref_weights = scaled_dot_product_attention(
    Q_test, K_test, V_test)

print(f"Output shape correct: {your_output.shape == ref_output.shape}")
print(f"Values match: {torch.allclose(your_output, ref_output, atol=1e-5)}")
print(f"Weights sum to 1: {torch.allclose(your_weights.sum(-1), torch.ones(2, 10), atol=1e-5)}")

if torch.allclose(your_output, ref_output, atol=1e-5):
    print("\nPerfect! Your implementation is correct!")
else:
    print("\nNot quite -- check your steps and try again.")

In [ ]:
#@title 🎧 Listen: Todo2
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_todo2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 2: Implement the TransformerBlock Forward Pass

We give you the `__init__` method fully defined. Your task is to implement the `forward` method.

In [ ]:
class TransformerBlockV2(nn.Module):
    """Your implementation of the Transformer encoder block."""
    def __init__(self, embed_dim, num_heads, mlp_ratio=4):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mhsa = MultiHeadAttention(embed_dim, num_heads)
        self.mlp = MLP(embed_dim, mlp_ratio)

    def forward(self, x):
        """
        Implement the pre-norm Transformer block.

        Steps:
            1. Apply self.norm1 to x
            2. Pass through self.mhsa -> (output, attn_weights)
            3. Add residual: add ORIGINAL x back
            4. Apply self.norm2 to the result
            5. Pass through self.mlp
            6. Add residual again
            7. Return final output and attn_weights
        """
        # TODO: ~6 lines of code
        pass

**Verification cell** — run this after you implement the forward method.

In [ ]:
# Verification for TODO 2
torch.manual_seed(42)
block_v2 = TransformerBlockV2(embed_dim=128, num_heads=4)
x_test = torch.randn(1, 65, 128)

output_v2, weights_v2 = block_v2(x_test)

print(f"Output shape: {output_v2.shape} (expected: [1, 65, 128])")
print(f"Shape correct: {output_v2.shape == torch.Size([1, 65, 128])}")
print(f"Number of heads: {len(weights_v2)} (expected: 4)")

diff = (output_v2 - x_test).abs().mean().item()
not_identical = not torch.allclose(output_v2, x_test, atol=1e-6)
print(f"Output differs from input: {not_identical}")

if output_v2.shape == torch.Size([1, 65, 128]) and not_identical:
    print("\nYour TransformerBlock implementation looks correct!")
else:
    print("\nSomething is off -- check your residual connections.")

In [ ]:
#@title 🎧 Listen: Todo3
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_todo3.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 3 (Stretch): Implement Causal Masking

This is optional, but it connects to how GPT-style models work. In causal attention, each token can only attend to tokens that come *before* it (and itself). This is enforced by masking future positions with $-\infty$ before softmax.

In [ ]:
def create_causal_mask(seq_len):
    """
    Create a causal attention mask.

    Returns:
        mask: Shape (seq_len, seq_len)
        mask[i][j] = 1 if j <= i, 0 if j > i

    Hint: Use torch.tril (lower triangular matrix)
    """
    # TODO: 1 line!
    pass


def causal_attention(Q, K, V):
    """
    Scaled dot-product attention with causal masking.

    Hint: scores.masked_fill(mask == 0, float('-inf'))
    """
    # TODO: Your implementation
    pass

In [ ]:
#@title 🎧 Listen: Putting Together
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/19_putting_together.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 6: Putting It All Together

Now let us stack multiple Transformer blocks to create a mini Transformer encoder, and feed real image patches through it.

First, we need to re-implement `PatchEmbedding` from Notebook 1 so this notebook is self-contained.

In [ ]:
class PatchEmbedding(nn.Module):
    """Patch embedding from Notebook 1. Self-contained."""
    def __init__(self, img_size=32, patch_size=4,
                 in_channels=3, embed_dim=128):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        patch_dim = in_channels * patch_size * patch_size
        self.projection = nn.Linear(patch_dim, embed_dim)
        self.cls_token = nn.Parameter(
            torch.randn(1, 1, embed_dim) * 0.02)
        self.pos_embed = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim) * 0.02)

    def forward(self, x):
        B, p = x.shape[0], self.patch_size
        patches = x.unfold(2, p, p).unfold(3, p, p)
        patches = patches.contiguous().view(B, 3, -1, p*p)
        patches = patches.permute(0, 2, 1, 3).contiguous()
        patches = patches.view(B, -1, 3 * p * p)
        emb = self.projection(patches)
        cls = self.cls_token.expand(B, -1, -1)
        emb = torch.cat([cls, emb], dim=1)
        return emb + self.pos_embed

print("PatchEmbedding defined!")
print(f"32x32 with 4x4 patches: {(32//4)**2} patches + 1 CLS = 65 tokens")

Now let us create a small Transformer encoder by stacking 4 blocks.

In [ ]:
class MiniViTEncoder(nn.Module):
    """Small Vision Transformer encoder for demonstration."""
    def __init__(self, img_size=32, patch_size=4, in_channels=3,
                 embed_dim=128, num_heads=4, num_layers=4,
                 mlp_ratio=4):
        super().__init__()
        self.patch_embed = PatchEmbedding(
            img_size, patch_size, in_channels, embed_dim)
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio)
            for _ in range(num_layers)])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x, return_attention=False):
        z = self.patch_embed(x)
        all_attention = []
        for block in self.blocks:
            z, attn_w = block(z)
            if return_attention:
                all_attention.append(attn_w)
        z = self.norm(z)
        if return_attention:
            return z, all_attention
        return z

print("MiniViTEncoder defined!")

Let us build the model and print a summary.

In [ ]:
torch.manual_seed(42)
model = MiniViTEncoder(
    img_size=32, patch_size=4, in_channels=3,
    embed_dim=128, num_heads=4, num_layers=4)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model architecture:")
print(f"  Image size:    32 x 32")
print(f"  Patch size:    4 x 4")
print(f"  Num patches:   64 (+ 1 CLS = 65 tokens)")
print(f"  Embed dim:     128")
print(f"  Num heads:     4 (head_dim = 32)")
print(f"  Num layers:    4")
print(f"  MLP ratio:     4 (hidden_dim = 512)")
print(f"  Total params:  {total_params:,}")
print(f"\nTiny model (~{total_params/1e6:.1f}M params).")
print("The original ViT-Base has 86M params!")

In [ ]:
#@title 🎧 Listen: Shape Trace
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/20_shape_trace.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us load a real CIFAR-10 image and pass it through the encoder.

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)

image, label = dataset[0]
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
print(f"Image class: {class_names[label]}")
print(f"Image shape: {image.shape}")

Let us trace the shapes through every stage of the forward pass.

In [ ]:
x = image.unsqueeze(0)  # Add batch dimension: (1, 3, 32, 32)
print(f"Input image:      {x.shape}")

with torch.no_grad():
    z = model.patch_embed(x)
    print(f"After patch embed: {z.shape}  (65 tokens x 128 dim)")

    for i, block in enumerate(model.blocks):
        z, attn_w = block(z)
        print(f"After block {i+1}:     {z.shape}  "
              f"({len(attn_w)} heads)")

    z = model.norm(z)
    print(f"After final norm:  {z.shape}")
    cls_output = z[:, 0]
    print(f"\n[CLS] token:       {cls_output.shape}")
    print("This vector would be fed to a classification head.")

In [ ]:
#@title 🎧 Listen: Cls Evolution
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/21_cls_evolution.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Let us verify that the [CLS] token representation changes across layers — showing that each layer refines the representation.

In [ ]:
with torch.no_grad():
    z = model.patch_embed(x)
    cls_reps = [z[:, 0].clone()]

    for block in model.blocks:
        z, _ = block(z)
        cls_reps.append(z[:, 0].clone())

# Cosine similarity between consecutive layers
print("Cosine similarity of [CLS] between consecutive layers:")
print("=" * 50)
for i in range(len(cls_reps) - 1):
    sim = F.cosine_similarity(
        cls_reps[i], cls_reps[i+1], dim=-1).item()
    src = "Embed" if i == 0 else f"Block {i}"
    print(f"  {src:>8} -> Block {i+1}: {sim:.4f}")

print("\nValues near 1.0 = small changes (residual at work).")
print("Values further from 1.0 = larger transformations.")

In [ ]:
#@title 🎧 Listen: Final Output Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/22_final_output_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 7: Final Output

This is the culmination of our work. We will feed CIFAR-10 images through our mini ViT encoder and create detailed attention visualizations that show how different heads and layers attend to different parts of the image.

In [ ]:
def show_image_with_patches(ax, image_tensor, patch_size=4, title=''):
    """Display an image with patch grid overlay."""
    img = image_tensor.permute(1, 2, 0).numpy() * 0.5 + 0.5
    img = np.clip(img, 0, 1)
    ax.imshow(img)
    H, W = img.shape[:2]
    for i in range(0, H + 1, patch_size):
        ax.axhline(y=i - 0.5, color='white', linewidth=0.5, alpha=0.5)
    for j in range(0, W + 1, patch_size):
        ax.axvline(x=j - 0.5, color='white', linewidth=0.5, alpha=0.5)
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.axis('off')

print("show_image_with_patches() defined!")

In [ ]:
def visualize_attention_on_image(ax, image_tensor, attn_weights,
                                  patch_size=4, token_idx=0, title=''):
    """Overlay attention weights from a specific token onto the image."""
    img = image_tensor.permute(1, 2, 0).numpy() * 0.5 + 0.5
    img = np.clip(img, 0, 1)
    H, W = img.shape[:2]
    grid_h, grid_w = H // patch_size, W // patch_size

    # Get attention to patch tokens (skip CLS at index 0)
    attn = attn_weights[token_idx, 1:].detach().numpy()
    attn_map = attn.reshape(grid_h, grid_w)

    # Normalize to [0, 1]
    attn_map = (attn_map - attn_map.min()) / \
               (attn_map.max() - attn_map.min() + 1e-8)

    # Upsample to image size
    attn_resized = np.kron(attn_map, np.ones((patch_size, patch_size)))

    ax.imshow(img)
    ax.imshow(attn_resized, alpha=0.6, cmap='hot')
    ax.set_title(title, fontsize=10, fontweight='bold')
    ax.axis('off')

print("visualize_attention_on_image() defined!")

Let us select a few interesting images from different classes for our visualization.

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

# Find images of different classes
sample_indices = []
seen_classes = set()
for i in range(len(dataset)):
    _, lbl = dataset[i]
    if lbl not in seen_classes and lbl in [3, 5, 0, 8]:
        sample_indices.append(i)
        seen_classes.add(lbl)
    if len(sample_indices) == 4:
        break

img_tensor, img_label = dataset[sample_indices[0]]
print(f"Visualizing attention for: {class_names[img_label]}")

In [ ]:
# Get attention weights from all layers
with torch.no_grad():
    output, all_attention = model(
        img_tensor.unsqueeze(0), return_attention=True)

print(f"Number of layers: {len(all_attention)}")
print(f"Heads per layer: {len(all_attention[0])}")
print(f"Attention shape per head: {all_attention[0][0].shape}")

In [ ]:
#@title 🎧 Listen: Cls Attention Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/23_cls_attention_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us create the first panel: original image alongside CLS attention from early and late layers.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

show_image_with_patches(
    axes[0], img_tensor, patch_size=4,
    title=f'Original: {class_names[img_label]}\n(8x8 grid)')

visualize_attention_on_image(
    axes[1], img_tensor,
    all_attention[0][0][0],  # Layer 0, Head 0, Batch 0
    patch_size=4, token_idx=0,
    title='Layer 1, Head 1\n[CLS] Attention')

visualize_attention_on_image(
    axes[2], img_tensor,
    all_attention[3][0][0],  # Layer 3, Head 0, Batch 0
    patch_size=4, token_idx=0,
    title='Layer 4, Head 1\n[CLS] Attention')

fig.suptitle('How the [CLS] Token Attends to Image Patches',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: All Heads Grid
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/24_all_heads_grid.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us create the full grid showing all heads across two layers — this reveals how different heads specialize.

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 9))
fig.suptitle(
    f'Attention Across Heads and Layers — '
    f'{class_names[img_label].upper()}\n'
    '[CLS] token attention to all image patches',
    fontsize=15, fontweight='bold', y=1.02)

for layer_idx, layer in enumerate([0, 3]):
    for head_idx in range(4):
        ax = axes[layer_idx, head_idx]
        attn = all_attention[layer][head_idx][0]
        visualize_attention_on_image(
            ax, img_tensor, attn,
            patch_size=4, token_idx=0,
            title=f'Layer {layer+1}, Head {head_idx+1}')

axes[0, 0].set_ylabel('Layer 1\n(Early)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Layer 4\n(Late)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

Let us also look at the raw attention matrices (not overlaid on images) to see the full token-to-token structure.

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20, 9))
fig.suptitle('Raw Attention Matrices — Full 65x65',
             fontsize=14, fontweight='bold', y=1.02)

for layer_idx, layer in enumerate([0, 3]):
    for head_idx in range(4):
        ax = axes[layer_idx, head_idx]
        attn = all_attention[layer][head_idx][0].detach().numpy()
        im = ax.imshow(attn, cmap='viridis', aspect='auto')
        ax.set_title(f'Layer {layer+1}, Head {head_idx+1}',
                     fontsize=11, fontweight='bold')
        if head_idx == 0:
            ax.set_ylabel(f'Layer {layer+1}', fontsize=11)
        if layer_idx == 1:
            ax.set_xlabel('Key position', fontsize=10)
        plt.colorbar(im, ax=ax, shrink=0.7)

plt.tight_layout()
plt.show()

Token 0 is [CLS]. Tokens 1-64 are image patches. Look for structure: diagonal patterns suggest local attention, uniform rows suggest global attention, bright spots suggest focused attention.

In [ ]:
#@title 🎧 Listen: Cross Image Comparison
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/25_cross_image_comparison.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Finally, let us compare attention across multiple images to see how the model adapts its attention to different content.

In [ ]:
n_images = len(sample_indices)
fig, axes = plt.subplots(n_images, 5, figsize=(22, 4 * n_images))
fig.suptitle(
    'Attention Comparison Across Different Images\n'
    '[CLS] attention from Layer 4 (deepest layer)',
    fontsize=15, fontweight='bold', y=1.02)

for row, idx in enumerate(sample_indices):
    img, lbl = dataset[idx]
    with torch.no_grad():
        _, attn_all = model(img.unsqueeze(0), return_attention=True)

    show_image_with_patches(
        axes[row, 0], img, patch_size=4,
        title=f'{class_names[lbl]}')

    for head in range(4):
        visualize_attention_on_image(
            axes[row, head + 1], img,
            attn_all[3][head][0],
            patch_size=4, token_idx=0,
            title=f'Head {head + 1}')

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 60)
print("  You have built a complete Transformer encoder for images!")
print("=" * 60)
print()
print("What we built from scratch in this notebook:")
print("  1. Scaled dot-product attention")
print("  2. Single attention head with learnable projections")
print("  3. Multi-head attention (4 heads)")
print("  4. MLP block with GELU and 4x expansion")
print("  5. Full Transformer encoder block (pre-norm)")
print("  6. Stacked 4 blocks into a mini ViT encoder")
print("  7. Visualized attention patterns on real images")
print()
print("Key insight: Even with random weights, the attention")
print("mechanism has the STRUCTURE to learn meaningful patterns.")
print("After training, each head specializes, and deeper layers")
print("capture increasingly semantic relationships.")

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/26_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## Section 8: Reflection and Next Steps

### Reflection Questions

Take a moment to think about these questions. Better yet, modify the code above to find out!

**1. Why does the scaling factor $1/\sqrt{d_k}$ matter?**
What would happen without it? Try removing the `/ math.sqrt(d_k)` from our attention function and see what happens to the attention weights. You should find that softmax produces nearly one-hot distributions — almost all the weight goes to a single token. This kills the ability to blend information from multiple tokens and makes gradients vanish.

**2. Each attention head has its own $W^Q$, $W^K$, $W^V$. What if all heads shared the same weights?**
Try modifying `MultiHeadAttention` so all heads share one set of projection matrices. What happens to the diversity of attention patterns in the visualization? You should see all heads produce nearly identical patterns, losing the benefit of multi-head attention.

**3. The residual connection adds the input back. What happens if you remove it?**
Try removing the `+ x` from the Transformer block. Train or evaluate — you will likely find that the model becomes much harder to train and the representations degrade in deeper layers. Residual connections are essential for gradient flow in deep networks.

### Optional Challenges

**Challenge 1: Learnable Temperature**
Instead of dividing by the fixed $\sqrt{d_k}$, make the temperature a learnable parameter. Initialize it to $\sqrt{d_k}$ and let it be updated during training. Does this help the model learn better attention patterns?

In [ ]:
# Hint: Add this to your attention head
# self.temperature = nn.Parameter(torch.tensor(math.sqrt(d_k)))
# Then use: scores = Q @ K.T / self.temperature

**Challenge 2: Visualize CLS Token Evolution with PCA**
The [CLS] token representation changes at each layer. Use PCA to project each layer's [CLS] representation to 2D and plot the trajectory.

In [ ]:
# Hint:
# from sklearn.decomposition import PCA
# Collect CLS representations from each layer
# Stack them: shape (num_layers+1, embed_dim)
# Fit PCA with n_components=2
# Plot the 2D trajectory with arrows

**Challenge 3: Relative Position Bias (Swin Transformer)**
The original ViT uses absolute positional embeddings (added once at the start). The Swin Transformer instead uses a relative position bias added directly to the attention scores. Implement a learnable relative position bias table.

In [ ]:
# Hint:
# The relative position bias B has shape (seq_len, seq_len)
# and gets ADDED to attention scores before softmax.
# B[i][j] depends on relative position of patch i and j.
# Use a learnable lookup table indexed by
# (2*grid_h - 1) * (2*grid_w - 1) possible relative positions.

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/27_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### What is Next?

In **Notebook 3**, we will put everything together into a complete Vision Transformer:
- Combine patch embedding (Notebook 1) + Transformer encoder (Notebook 2)
- Add the classification head
- Train on CIFAR-10 and watch the attention patterns evolve during training
- Compare performance to a simple CNN baseline
- Implement class attention maps to see what the model has learned

The pieces are all in place. Notebook 3 is where we train and see the full power of the Vision Transformer!